In [ ]:
from __future__ import unicode_literals, print_function


import os
from pathlib import Path
import spacy
import plac

model_dir = './Models/'
test_dir = './Data/TestData/'
output_dir = './Output/'

nlp = spacy.load(model_dir)

## Read the abstracts

Read the abstracts that are located at the `/Data/TestData/` folder.

In [43]:
import nltk

abstracts = []
sentences = []
ent_tagged_text = []

def tag_entity_text(text, replacementText):
    newString1 = ""
    print(text)
    for (t,r) in zip(text,replacementText):
        newString1 = text.replace(t,r)
        sentence = newString1
    return sentence

with open('./Data/TestData/test_p31_intro.txt') as file:
    lines = file.readlines()
    abstxt = ''.join(str(line) for line in lines)
    abstxt = abstxt.lower()
    abstracts.append(abstxt)
    

# tokenize the abstracts into a sentence
for abstract in abstracts:
    sents = nltk.sent_tokenize(abstract)
    for sent in sents:
        sentences.append(sent)

        
text = []
replacementText = []
semeval_tagged = []
for sentence in sentences:
#     print(sentence + " : : ")
    ner_tagged = nlp(sentence)
    tagged_entities = ner_tagged.ents
    tuple_length = len(tagged_entities)
    text = []
    replacementText = []
    if(tuple_length == 2):
        for (i,ent) in enumerate(tagged_entities):
            print(str(i) + ent.text)
            text.append(ent.text)
            replacementText.append('<e' + str(i+1) + '>' + ent.text + '</e' + str(i+1) + '>')
            semevalify = tag_entity_text(text,replacementText)
            semeval_tagged.append(semevalify)
            

print(semeval_tagged)

0jacobian matrix


AttributeError: 'list' object has no attribute 'replace'

In [ ]:
for ent in nlp(sentence).ents:
        labels = ent.label_
        print("%s %s %d %d\n" % (ent.label_.encode("utf-8"), ent.text.encode("utf-8"), ent.start_char, ent.end_char))